For this lab, you are going to be combining the fare data you used in **Lab 1** with some trip data about those same exact taxi rides.

Here is the schema of the trip dataset, found in `trip_data_1_trimmed.csv`:

* `medallion`: The ID of the cab being operated
* `hack_license`: The ID of the person operating the cab
* `vendor_id`: The type of vendor operating the cab, can either be `CMT` or `VTS`, no clue what these two types mean
* `rate_code`: Designates the kind of ride this is, must be `1` through `6`, any other number is incorrect
* `store_and_fwd_flag`: Can be either `Y`,`N`, or Nan
* `pickup_datetime`: The time when the ride started
* `dropoff_datetime`: The time when the ride ended
* `passenger_count`: The number of passengers during the ride
* `trip_time_in_secs`: How long the trip took
* `trip_distance`: Distance of the trip, to the nearest 1/10 mile
* `pickup_longitude`: Longitude of pickup location
* `pickup_latitude`: Latitude of pickup location
* `dropoff_longitude`: Longitude of dropoff location
* `dropoff_latitude`: Latitude of dropoff location

And here is the schema for the fare dataset, as a refresher (you're going to be using it too :)), foud in `fare_data_1_trimmed.csv`:

* `medallion`: The ID of the cab being operated
* `hack_license`: The ID of the person operating the cab
* `vendor_id`: The type of vendor operating the cab, can either be `CMT` or `VTS`, no clue what these two types mean
* `pickup_datetime`: The time when the ride started
* `payment_type`: How the trip was paid, `UNK` stands for unknown, I have no idea what `NOC` stands for, but lets assume its some known way to pay
* `fare_amount`: Base fare cost of the trip
* `surcharge`: Additional charges that are not tolls
* `mta_tax`: The mta has to get its cut, right? :)
* `tip_amount`: How generous the rider(s) decided to be
* `tolls_amount`: How much money you had to pay in tolls
* `total_amount`: How much the trip cost, all in

One final piece of information you will need, the approximate latitude/longitude bounds of a bounding box centered around each borough in NYC:

* Queens,40.800760,40.542920,-73.700272,-73.962616
* Manhattan,40.874663,40.701293,-73.910759,-74.018721
* Bronx,40.915255,40.785743,-73.765274,-73.933406
* Brooklyn,40.739877,40.57042,-73.864754,-74.04344
* Staten Island,40.651812,40.477399,-74.034547,-74.259090

So, as was the case with the last lab, you are going to be tasked with answering a whole slew of questions about these data, except the questions here should be significantly more challenging.

Before beginning, please:

1. Remove any rides that did not have an appropriate `rate_code` 
* Convert pickup and dropoff locations from latitude/longitude to (approximate) borough. This will be very challening. I suggest the following approach, although you can try others:
  1. Calculate the eaxct center of each borough's bounding box
  * Find the distance from the given ride's pickup/dropoff location and the center of each borough found in A. 
  * Pick the borough that had the smallest distance from the given location
* Once this is done, join the `trip_data` and `fare_data` datasets together (you will need to join the datasets on more than one column).

Once you've gotten the preprocessing out of the way, answer the following questions:

1. What was the most common borough start location?
  * End location?
  * Pair of start/end locations? Excluding manhattan/manhattan?
* Which driver (`hack_license`) carried the most passengers, on average?
* Was there any relationship between (correlation) when a taxi ride ended (split the day into quarters) and the tip percentage on the fare?
* Did the trip time correlate with the cost of the trip?
  * What about the tip amount?
* Which cab tended to generate the most revenue, on average, when picking people up in manhattan?
* What was the average cost of all of the trips that originated in Brooklyn?
* What was the average trip distance for intra-borough trips?
  1. For inter-borough trips?

In [36]:
import pandas as pd

In [37]:
fareData = pd.read_csv("./nycTaxiData/trip_fare_1_trimmed.csv")
tripData = pd.read_csv("./nycTaxiData/trip_data_1_trimmed.csv")

In [40]:
tripData.head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,89D227B655E5C82AECF13C3F540D4CF4,BA96DE419E711691B9445D6A6307C170,CMT,1,N,2013-01-01 15:11:48,2013-01-01 15:18:10,4,382,1.0,-73.978165,40.757977,-73.989838,40.751171
1,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-06 00:18:35,2013-01-06 00:22:54,1,259,1.5,-74.006683,40.731781,-73.994499,40.750660
2,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-05 18:49:41,2013-01-05 18:54:23,1,282,1.1,-74.004707,40.737770,-74.009834,40.726002
3,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:54:15,2013-01-07 23:58:20,2,244,0.7,-73.974602,40.759945,-73.984734,40.759388
4,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:25:03,2013-01-07 23:34:24,1,560,2.1,-73.976250,40.748528,-74.002586,40.747868


array(['CMT', 'VTS'], dtype=object)